In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
from datetime import datetime

print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
from DateProvider import DateProvider
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
def getWorkingDirectory():
    pwd = ! "pwd"
    return pwd[0]

In [ ]:
pwd = getWorkingDirectory()

In [ ]:
pwd

## Download VAERS-Data

In [ ]:
from VAERSFileDownloader import downloadVAERSFileAndUnzip

In [ ]:
if needsUpdate:
    workingDirectory = getWorkingDirectory()
    downloadVAERSFileAndUnzip('2022VAERSData.zip', workingDirectory)
    downloadVAERSFileAndUnzip('NonDomesticVAERSData.zip', workingDirectory)

In [ ]:
from DataFrameFilter import DataFrameFilter

In [ ]:
from BatchCodeTableFactory import BatchCodeTableFactory

In [ ]:
from BatchCodeTableHtmlUpdater import saveCountryOptions, saveLastUpdatedBatchCodeTable

In [ ]:
from VaersReader import getVaersForYears, getNonDomesticVaers

In [ ]:
internationalVaers = pd.concat([getVaersForYears([2020, 2021, 2022]), getNonDomesticVaers()])
internationalVaersCovid19 = DataFrameFilter().filterByCovid19(internationalVaers)
internationalVaersCovid19

In [ ]:
from BatchCodeTablePersister import createAndSaveBatchCodeTableForCountry, createAndSaveBatchCodeTablesForCountries

In [ ]:
from HtmlUtils import getCountryOptions

In [ ]:
countries = sorted(internationalVaersCovid19['COUNTRY'].unique())
countryOptions = getCountryOptions(countries)


In [ ]:
saveCountryOptions(countryOptions)

In [ ]:
saveLastUpdatedBatchCodeTable(dateProvider.getLastUpdatedDataSource())

In [ ]:
minADRsForLethality = 100
batchCodeTableFactory = BatchCodeTableFactory(internationalVaersCovid19)

createAndSaveBatchCodeTablesForCountries(
    createBatchCodeTableForCountry = lambda country: batchCodeTableFactory.createBatchCodeTableByCountry(country),
    countries = countries,
    minADRsForLethality = minADRsForLethality)

createAndSaveBatchCodeTableForCountry(
    createBatchCodeTableForCountry = lambda country: batchCodeTableFactory.createGlobalBatchCodeTable(),
    country = 'Global',
    minADRsForLethality = minADRsForLethality)

In [ ]:
def publishGitHubPages():
    %cd /home/frankknoll/Dokumente/Corona/projects/HowBadIsMyBatch-pages
    ! git add -A
    ! git commit -m "updating data for batch codes"
    ! git push

### see https://knollfrank.github.io/HowBadIsMyBatch/batchCodeTable.html

In [ ]:
publishGitHubPages()